In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# /Users/chriswesterman/Library/Jupyter/nbextensions/snippets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import beatthebookies.data as btb

pd.set_option('display.max_columns', 500)


# Exploratory notebook

In [14]:
matches = btb.get_data()

get_data 6.8


In [15]:
matches.head()

,id,season,stage,date,home_team,away_team,home_team_goal,away_team_goal,home_w,away_w,draw,home_t_home_goals,home_t_total_goals,home_t_home_goals_against,home_t_total_goals_against,home_t_home_wins,home_t_home_losses,home_t_home_draws,home_t_total_wins,home_t_total_losses,home_t_total_draws,away_t_away_goals,away_t_total_goals,away_t_away_goals_against,away_t_total_goals_against,away_t_away_wins,away_t_away_losses,away_t_away_draws,away_t_total_wins,away_t_total_losses,away_t_total_draws
1,4390,2015/2016,1,2015-08-08 00:00:00,Bournemouth,Aston Villa,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4391,2015/2016,1,2015-08-08 00:00:00,Chelsea,Swansea City,2,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4392,2015/2016,1,2015-08-08 00:00:00,Everton,Watford,2,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4393,2015/2016,1,2015-08-08 00:00:00,Leicester City,Sunderland,4,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,4394,2015/2016,1,2015-08-08 00:00:00,Manchester United,Tottenham Hotspur,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
matches = btb.get_totals(matches)

get_totals 6.56


In [10]:
matches.tail(10)

,id,season,stage,date,home_team,away_team,home_team_goal,away_team_goal,home_w,away_w,draw,home_t_home_goals,home_t_total_goals,home_t_home_goals_against,home_t_total_goals_against,home_t_home_wins,home_t_home_losses,home_t_home_draws,home_t_total_wins,home_t_total_losses,home_t_total_draws,away_t_away_goals,away_t_total_goals,away_t_away_goals_against,away_t_total_goals_against,away_t_away_wins,away_t_away_losses,away_t_away_draws,away_t_total_wins,away_t_total_losses,away_t_total_draws
319,4708,2015/2016,38,2016-05-15 00:00:00,West Bromwich Albion,Liverpool,1,1,0,0,1,19,33,25,41,6,8,4,10,15,12,29,62,27,51,8,7,3,16,10,11
318,4707,2015/2016,38,2016-05-15 00:00:00,Watford,Sunderland,2,2,0,0,1,18,38,17,49,6,7,5,12,17,8,23,46,40,43,3,10,5,9,17,11
317,4706,2015/2016,38,2016-05-15 00:00:00,Swansea City,Manchester City,1,1,0,0,1,19,41,19,51,8,5,5,12,15,10,23,70,19,44,7,5,6,19,10,8
316,4705,2015/2016,38,2016-05-15 00:00:00,Stoke City,West Ham United,2,1,1,0,0,20,39,23,51,7,7,4,13,15,9,30,64,23,56,7,4,7,16,7,14
314,4703,2015/2016,38,2016-05-15 00:00:00,Newcastle United,Tottenham Hotspur,5,1,1,0,0,27,39,23,68,6,5,7,8,19,10,33,68,15,48,9,2,7,19,5,13
312,4701,2015/2016,38,2016-05-15 00:00:00,Everton,Norwich City,3,0,1,0,0,32,56,30,57,5,8,5,10,13,14,13,39,34,43,3,13,2,9,21,7
311,4700,2015/2016,38,2016-05-15 00:00:00,Chelsea,Leicester City,1,1,0,0,1,31,58,29,54,5,5,8,12,12,13,32,67,17,50,11,2,5,23,3,11
310,4699,2015/2016,38,2016-05-15 00:00:00,Arsenal,Aston Villa,4,0,1,0,0,27,61,11,52,11,3,4,19,7,11,13,27,37,48,1,14,3,3,26,8
315,4704,2015/2016,38,2016-05-15 00:00:00,Southampton,Crystal Palace,4,1,1,0,0,35,55,21,54,10,5,3,17,11,9,19,38,24,42,5,7,6,11,17,9
313,4702,2015/2016,38,2016-05-17 00:00:00,Manchester United,Bournemouth,3,1,1,0,0,24,46,8,50,11,2,5,18,10,9,21,44,30,55,6,8,4,11,17,9


In [ ]:
path = "../data/"  #Insert path here
database = path + 'database.sqlite'

conn = sqlite3.connect(database)


In [ ]:
matches = pd.read_sql("""SELECT m.id, 
                        m.season, m.stage, m.date, 
                        ht.team_long_name as home_team, at.team_long_name as away_team, m.home_team_goal, 
                        m.away_team_goal                                      
                        FROM Match as m
                        LEFT JOIN Team AS ht on ht.team_api_id = m.home_team_api_id
                        LEFT JOIN Team AS at on at.team_api_id = m.away_team_api_id
                        WHERE league_id = :league AND season = :season
                        ;""", conn, params={"league":1729, "season": "2015/2016"})
pd.set_option('display.max_columns', 500)
matches.head()

In [ ]:
matches.sort_values('date', inplace=True)

In [ ]:
matches.head()

In [ ]:
matches['home_w'] = 0
matches['away_w'] = 0
matches['draw'] = 0

matches.loc[matches['home_team_goal'] > matches['away_team_goal'], 'home_w'] = 1
matches.loc[matches['home_team_goal'] < matches['away_team_goal'], 'away_w'] = 1
matches.loc[matches['home_team_goal'] == matches['away_team_goal'], 'draw'] = 1

In [ ]:
matches['home_t_home_goals'] = matches.groupby('home_team')['home_team_goal'].apply(lambda x  : x.cumsum().shift(fill_value=0))
matches['home_t_total_goals'] = 0

matches['home_t_goals_against'] = matches.groupby('home_team')['away_team_goal'].apply(lambda x  : x.cumsum().shift(fill_value=0))
# matches['home_t_total_goals_against'] = 0

matches['home_t_home_wins'] = matches.groupby('home_team')['home_w'].apply(lambda x  : x.cumsum().shift(fill_value=0))
matches['home_t_home_losses'] = matches.groupby('home_team')['away_w'].apply(lambda x  : x.cumsum().shift(fill_value=0))
matches['home_t_home_draws'] = matches.groupby('home_team')['draw'].apply(lambda x  : x.cumsum().shift(fill_value=0))
# matches['home_t_total_wins'] = 0
# matches['home_t_total_losses'] = 0
# matches['home_t_total_draws'] = 0


matches['away_t_away_goals'] = matches.groupby('away_team')['away_team_goal'].apply(lambda x  : x.cumsum().shift(fill_value=0))
matches['away_t_total_goals'] = 0
matches['away_t_away_goals_against'] = matches.groupby('away_team')['home_team_goal'].apply(lambda x  : x.cumsum().shift(fill_value=0))
# matches['away_t_total_goals_against'] = 0


matches['away_t_away_wins'] = matches.groupby('away_team')['away_w'].apply(lambda x  : x.cumsum().shift(fill_value=0))
matches['away_t_away_losses'] = matches.groupby('away_team')['home_w'].apply(lambda x  : x.cumsum().shift(fill_value=0))
matches['away_t_away_draws'] = matches.groupby('away_team')['draw'].apply(lambda x  : x.cumsum().shift(fill_value=0))
# matches['away_t_total_wins'] = 0
# matches['away_t_total_losses'] = 0
# matches['away_t_total_draws'] = 0

In [ ]:
matches.head()

In [ ]:
def ht_total_goals(st, ht, hg):
    if st == 1:
        return 0
    total_goals = matches[(matches['stage'] < st) & (matches['away_team'] == ht)]\
        .groupby('away_team')['away_team_goal'].sum() + hg
    return total_goals[0]
    
    
    

def at_total_goals(st, at, ag):
    if st == 1:
        return 0
    total_goals = matches[(matches['stage'] < st) & (matches['home_team'] == at)]\
        .groupby('home_team')['home_team_goal'].sum() + ag
    return total_goals[0]


matches['home_t_total_goals'] = matches.apply(lambda x: ht_total_goals( x['stage'], x['home_team'], x['home_t_home_goals']), axis=1)

matches['away_t_total_goals'] = matches.apply(lambda x: at_total_goals( x['stage'], x['away_team'], x['away_t_away_goals']), axis=1)



In [ ]:
matches[(matches.home_team =='Arsenal') | (matches.away_team == 'Arsenal')]

In [ ]:
matches[(matches['stage'] < 4) & (matches['away_team'] == 'Everton' )].groupby('away_team')['away_team_goal'].sum()

In [ ]:
for index, df in matches.iterrows():
    total_goals = matches[(matches['stage'] < df['stage']) & (matches['away_team'] == df['home_team'])].groupby('away_team')['away_team_goal'].sum() + df['home_t_home_goals']
    print(total_goals)

In [ ]:
test = pd.DataFrame([[1,2,3],[1,5,6],[2,5,6],[2,2,1]], columns=['a', 'b', 'c'])

print(test)

def test_func(a,b):
    return a + b


test['d'] = test.apply(lambda x: test_func(x['a'], x['b']), axis=1)

print(test)